In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa

In [2]:
from dask import dataframe as dd 
from dask import array as da
from dask.diagnostics import ProgressBar # type: ignore

In [3]:
# read h5
embeddings = dd.read_hdf('embeddings.h5', key='embeddings', mode='r', chunksize=1000)

# load 10'000
# embeddings = embeddings.head(10000)

In [4]:
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.202381,0.087937,0.221610,0.171785,-0.782774,-0.325774,0.120273,0.625501,-0.072252,0.004225,...,0.421689,-0.342691,0.289151,-0.748062,-0.896266,0.605513,0.282313,0.264579,0.758309,0.718300
1,-0.202381,0.087936,0.221610,0.171785,-0.782773,-0.325774,0.120273,0.625501,-0.072252,0.004225,...,0.421690,-0.342690,0.289151,-0.748062,-0.896267,0.605513,0.282313,0.264579,0.758308,0.718299
2,-0.202380,0.087936,0.221610,0.171785,-0.782774,-0.325774,0.120273,0.625501,-0.072252,0.004225,...,0.421690,-0.342690,0.289151,-0.748062,-0.896267,0.605513,0.282313,0.264579,0.758308,0.718299
3,-0.202380,0.087936,0.221610,0.171785,-0.782773,-0.325774,0.120273,0.625501,-0.072252,0.004225,...,0.421690,-0.342690,0.289151,-0.748062,-0.896266,0.605513,0.282313,0.264579,0.758308,0.718299
4,-0.202380,0.087936,0.221611,0.171787,-0.782776,-0.325774,0.120273,0.625500,-0.072251,0.004225,...,0.421690,-0.342691,0.289150,-0.748062,-0.896269,0.605514,0.282313,0.264579,0.758308,0.718300


In [5]:
embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
npartitions=552,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,floa

In [6]:
embeddings.shape

(<dask_expr.expr.Scalar: expr=FromGraph(4429402).size() // 768, dtype=int64>,
 768)

In [7]:
from dask_ml.decomposition import PCA

pca = PCA(n_components=25, random_state=42, svd_solver='randomized') 

# fit transform
embeddings_pca = pca.fit_transform(embeddings)

In [8]:
pca.explained_variance_ratio_.cumsum()[-1]

np.float64(0.28896943674736747)

In [9]:
embeddings_pca.compute_chunk_sizes()

dask.array<mul, shape=(551657, 25), dtype=float64, chunksize=(1000, 25), chunktype=numpy.ndarray>

In [10]:
# compute 
embeddings_pca = embeddings_pca.compute()

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=150,
    init='random',
    n_init=5,
    verbose=1,
)

kmeans.fit(embeddings_pca)

Initialization complete
Iteration 0, inertia 7.155533702253486e-06.
Iteration 1, inertia 6.119226754053294e-06.
Iteration 2, inertia 5.714682724555707e-06.
Iteration 3, inertia 5.3945595967958564e-06.
Iteration 4, inertia 5.138950894061022e-06.
Iteration 5, inertia 5.051722698565639e-06.
Iteration 6, inertia 5.006216519985774e-06.
Iteration 7, inertia 4.97630746923798e-06.
Iteration 8, inertia 4.949719326087439e-06.
Iteration 9, inertia 4.937279415497818e-06.
Iteration 10, inertia 4.925940272648512e-06.
Iteration 11, inertia 4.914201418280866e-06.
Iteration 12, inertia 4.907003667814566e-06.
Iteration 13, inertia 4.896471264976485e-06.
Iteration 14, inertia 4.877358022006542e-06.
Iteration 15, inertia 4.8563624539507325e-06.
Iteration 16, inertia 4.844198372203949e-06.
Iteration 17, inertia 4.837771114047462e-06.
Iteration 18, inertia 4.8316739795836154e-06.
Iteration 19, inertia 4.823607396647987e-06.
Iteration 20, inertia 4.819351958716929e-06.
Iteration 21, inertia 4.816580777519477

KMeans(init='random', n_clusters=150, n_init=5, verbose=1)

In [12]:
cluster_labels = kmeans.labels_
cluster_labels

array([ 97, 109,  92, ..., 144,  80,   9], dtype=int32)

In [13]:
# debug clustering results
kmeans.cluster_centers_.shape

(150, 25)

In [14]:
kmeans.cluster_centers_[0]

array([-7.41771778e-07, -1.27373629e-06, -3.28355674e-07, -5.62919353e-07,
        1.29731722e-07,  6.62606169e-07,  4.11580747e-07,  4.00685954e-07,
        2.99119271e-07, -2.89069690e-07, -3.29972453e-08,  4.78571735e-08,
       -1.93893315e-07, -8.02225072e-08,  1.39250331e-07,  9.08043020e-08,
        1.13929968e-07, -1.28096173e-07,  1.23148060e-07, -3.56704435e-08,
        5.22761246e-08, -1.12930712e-07, -7.07049364e-09,  1.22920586e-08,
       -2.76331240e-08])

In [15]:
# save cluster centers as h5
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
cluster_centers.to_hdf('cluster_centers_3.h5', key='cluster_centers', mode='w')

In [16]:
import dask.dataframe as dd
import dask.array as da
from dask.array import random as da_random

In [17]:
subset_size = 10_000 
total_size = embeddings_pca.shape[0]

# take random subset of embeddings_pca
random_indices = da_random.choice(total_size, subset_size, chunks=50_000)
subset_embeddings = embeddings_pca[random_indices]

In [18]:
subset_embeddings.shape

(10000, 25)

In [19]:
from sklearn.manifold import Isomap

lle = Isomap(n_components=2, n_neighbors=30, n_jobs=-1)

lle.fit(subset_embeddings)

Isomap(n_jobs=-1, n_neighbors=30)

In [20]:
# isomap statistics
lle.dist_matrix_.shape

(10000, 10000)

In [21]:
lle.embedding_.shape

(10000, 2)

In [22]:
# error of the embedding
lle.reconstruction_error() #np.float64(3.714505863838574e-11)

np.float64(2.408058568792537e-11)

In [25]:
# apply to each row, chunk by chunk
tsne_embeddings = dd.from_array(embeddings_pca, chunksize=100)
tsne_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
npartitions=5517,,,,,,,,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
100,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551600,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551656,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [26]:
# enable progress bar
ProgressBar().register()

# transform using tsne to embeddings_pca chunk by chunk
def apply_tsne(chunk):
    return lle.transform(chunk)

tsne_embeddings = tsne_embeddings.map_partitions(apply_tsne).compute()

[                                        ] | 0% Completed | 10.74 sms
[                                        ] | 0% Completed | 10.75 s


MemoryError: Unable to allocate 2.29 MiB for an array with shape (30, 10000) and data type float64

In [ ]:
cluster_labels

array([ 56, 102,  94, ..., 121,  72,  91], dtype=int32)

In [ ]:
# save tsne embeddings
cluster_labels = pd.DataFrame(cluster_labels)

[                                        ] | 0% Completed | 544.86 ms

In [ ]:
tsne_embeddings

,0,1
0,0.983825,-0.172378
1,-0.376714,-1.357544
2,-1.278375,0.209579
3,-1.135117,0.217335
4,-1.036053,-1.464109
...,...,...
551652,-0.789648,0.430628
551653,-0.465950,2.318494
551654,-0.592554,-0.088504
551655,-0.751131,-1.080699


In [ ]:
# save to h5
tsne_embeddings.to_hdf('tsne_embeddings_3.h5', key='tsne_embeddings', mode='w', format='table')

In [ ]:
cluster_labels.to_hdf('clusters_3.h5', key='clusters', mode='w', format='table')